# IMDb Movie Data Frame Builder

## Import Necessary Packages

In [35]:
%matplotlib inline
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV

from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error
import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix, parallel_coordinates
import seaborn as sns
import numpy as np

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 20)

## Read in Data - Multiple datasets from IMDb's Non Commercial public Data Sets

In [36]:
imdbMovies_df = pd.read_csv("/home/cb072/CSCI349/csci349_2024fa/Final Project/IMDB Final Project - Charles Blanch/IMDB Model Data/title.basics.tsv", sep='\t', low_memory=False)
imdbRatings_df = pd.read_csv("/home/cb072/CSCI349/csci349_2024fa/Final Project/IMDB Final Project - Charles Blanch/IMDB Model Data/title.ratings.tsv", sep='\t', low_memory=False)

## Merge the Two Data Frames

In [37]:
# Merge the two data sets on the tconst column and drop duplicate columns
imdb_df = pd.merge(imdbMovies_df, imdbRatings_df, on='tconst')
imdb_df.drop(['originalTitle', 'endYear'], axis=1, inplace=True)

In [38]:
# Filter out movies with less than 50,000 votes and sort by rating
popular_movies_df = imdb_df[(imdb_df['titleType'] == 'movie') & (imdb_df['numVotes'] > 100000)].sort_values(by=['averageRating', 'numVotes'], ascending=False)
popular_movies_df.rename(columns={'tconst':'Const'}, inplace=True)

In [39]:
# Read in a new data set with the movie directors
director_df = pd.read_csv("/home/cb072/CSCI349/csci349_2024fa/Final Project/IMDB Final Project - Charles Blanch/IMDB Model Data/title.crew.tsv", sep='\t', usecols=['tconst', 'directors'])
director_df.rename(columns={'tconst': 'Const'}, inplace=True)

In [40]:
# Merge the directors into the popular movies data set
popular_movies_df = pd.merge(popular_movies_df, director_df, on='Const', how='left')

In [41]:
# Read in a new data set with the names of movie actors and directors
names_df = pd.read_csv("/home/cb072/CSCI349/csci349_2024fa/Final Project/IMDB Final Project - Charles Blanch/IMDB Model Data/name.basics.tsv", sep='\t', usecols=['nconst', 'primaryName'])

In [42]:
# Merge the names into the popular movies data set based on nconst
popular_movies_df = pd.merge(popular_movies_df, names_df, left_on='directors', right_on='nconst', how='left')

# Drop the columns that id columns
popular_movies_df.drop(['directors', 'nconst'], axis=1, inplace=True)

In [43]:
# Fill in missing director names with 'Multiple Directors'
popular_movies_df['primaryName'].fillna('Multiple Directors', inplace=True)

In [44]:
# Read in a new data set with user ratings - your own data can be used by downloading ratings off of IMDb website
my_ratings_df = pd.read_csv("/home/cb072/CSCI349/csci349_2024fa/Final Project/IMDB Final Project - Charles Blanch/IMDB Model Data/ratings.csv")
my_ratings_df = my_ratings_df[my_ratings_df['Title Type'] == 'Movie']


In [45]:
my_ratings_df

,Const,Your Rating,Date Rated,Title,Original Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors
0,tt0338348,7,2024-11-18,The Polar Express,The Polar Express,https://www.imdb.com/title/tt0338348,Movie,6.6,100.0,2004,"Animation, Musical, Adventure, Comedy, Family,...",247067,2004-11-10,Robert Zemeckis
1,tt0319343,7,2024-11-18,Elf,Elf,https://www.imdb.com/title/tt0319343,Movie,7.1,97.0,2003,"Family, Fantasy, Romance, Adventure, Comedy",315842,2003-11-07,Jon Favreau
2,tt7991608,5,2024-11-18,Red Notice,Red Notice,https://www.imdb.com/title/tt7991608,Movie,6.3,118.0,2021,"Action, Comedy, Thriller",329435,2021-11-12,Rawson Marshall Thurber
9,tt6263850,8,2024-07-27,Deadpool & Wolverine,Deadpool & Wolverine,https://www.imdb.com/title/tt6263850,Movie,7.7,128.0,2024,"Action, Comedy, Sci-Fi, Adventure",381381,2024-07-26,Shawn Levy
10,tt0338013,9,2024-07-22,Eternal Sunshine of the Spotless Mind,Eternal Sunshine of the Spotless Mind,https://www.imdb.com/title/tt0338013,Movie,8.3,108.0,2004,"Drama, Sci-Fi, Romance",1106236,2004-03-19,Michel Gondry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,tt1745960,8,2022-08-13,Top Gun: Maverick,Top Gun: Maverick,https://www.imdb.com/title/tt1745960,Movie,8.2,130.0,2022,"Action, Drama",740120,2022-05-27,Joseph Kosinski
229,tt0080684,9,2022-08-13,Star Wars: Episode V - The Empire Strikes Back,Star Wars: Episode V - The Empire Strikes Back,https://www.imdb.com/title/tt0080684,Movie,8.7,124.0,1980,"Fantasy, Sci-Fi, Adventure, Action",1410871,1980-06-18,Irvin Kershner
230,tt1375666,9,2022-08-13,Inception,Inception,https://www.imdb.com/title/tt1375666,Movie,8.8,148.0,2010,"Sci-Fi, Thriller, Action, Adventure",2614752,2010-07-16,Christopher Nolan
231,tt0468569,10,2022-08-13,The Dark Knight,The Dark Knight,https://www.imdb.com/title/tt0468569,Movie,9.0,152.0,2008,"Drama, Crime, Thriller, Action",2946567,2008-07-18,Christopher Nolan


In [46]:
# Merge the user ratings into the popular movies data set
imdb_df = pd.merge(popular_movies_df, my_ratings_df, on='Const', how='left')


In [47]:
# Drop duplicate columns from merge
imdb_df = imdb_df.drop(['Title', 'Title Type', 'IMDb Rating', "Runtime (mins)", "Year", "Genres", 'Num Votes', 'Release Date', 'URL', 'Directors'], axis = 1)

In [48]:
imdb_df['has_rated'] = imdb_df['Your Rating'].notnull()

In [49]:
#Rename columns for clarity
imdb_df.rename(columns={'averageRating': 'IMDb Rating', 'primaryName': 'Director', 'Your Rating': 'My Rating'}, inplace=True)

In [50]:
# Read in a new data set with the movie cast                                                                                                                 
interested_tconsts = set(imdb_df['Const'])

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Define the chunk size
chunk_size = 100000  # Adjust based on your system's memory capacity

# Read in chunks
for chunk in pd.read_csv("/home/cb072/CSCI349/csci349_2024fa/Final Project/IMDB Final Project - Charles Blanch/IMDB Model Data/title.principals.tsv", sep='\t', usecols=['tconst', 'nconst', 'ordering'], chunksize=chunk_size):
    # Filter each chunk for rows where tconst is in interested_tconsts
    filtered_chunk = chunk[chunk['tconst'].isin(interested_tconsts)]
    filtered_chunks.append(filtered_chunk)

# Concatenate the filtered chunks together
cast_df = pd.concat(filtered_chunks, ignore_index=True)

cast_df.rename(columns={'tconst': 'Const'}, inplace=True)

In [51]:
# Filters the cast data frame for only the movies that are in the complete data frame
filtered_cast_df = pd.merge(cast_df, imdb_df[['Const']], on='Const', how='inner')

# Filters the cast data frame for only the top 3 lead actors
lead_cast_df = filtered_cast_df[filtered_cast_df['ordering'] <= 3]

In [52]:
# Pivots the lead cast data frame to get the top 3 lead actors in separate columns for each movie
lead_cast_pivot = lead_cast_df.pivot(index='Const', columns='ordering', values='nconst').reset_index()

lead_cast_pivot.columns = ['Const', 'lead_cast_1', 'lead_cast_2', 'lead_cast_3']

In [53]:
# Merge the lead actor ids into the complete data frame
imdb_df = pd.merge(imdb_df, lead_cast_pivot, on='Const', how='left')

In [54]:
# Merge the names of the lead actors into the complete data frame
for i in range(1, 4):  # For lead_cast_1, lead_cast_2, lead_cast_3
    # Perform the merge operation with names_df to get names, but keep it separate
    temp_df = pd.merge(imdb_df[['Const', f'lead_cast_{i}']], names_df, left_on=f'lead_cast_{i}', right_on='nconst', how='left')
    
    # Now, update complete_df by adding the new name column
    imdb_df[f'lead_cast_{i}_name'] = temp_df['primaryName']

# Drop the nconst columns
imdb_df.drop(['lead_cast_1', 'lead_cast_2', 'lead_cast_3'], axis=1, inplace=True)

## Create new dummy and observational columns & Clean Dataframe

In [55]:
# Drop unnecessary columns - titleType is always 'movie', isAdult is always 0
imdb_df = imdb_df.drop(columns=["titleType", "isAdult"])

# Rename columns for clarity
imdb_df.rename(columns={"startYear": "ReleaseYear", "primaryTitle": "Title", "runtimeMinutes": "Runtime"}, inplace=True)

# Create an Age column in years
imdb_df["ReleaseYear"] = imdb_df["ReleaseYear"].astype(int)
imdb_df["Age"] = 2024 - imdb_df["ReleaseYear"]

# Create a column for the difference between my rating and IMDb rating
imdb_df["DifferenceBetweenRatings"] = imdb_df["My Rating"] - imdb_df["IMDb Rating"]

# Create a dummy column for if a movie is longer than 2 hours
imdb_df["Runtime"] = imdb_df["Runtime"].astype(int)
imdb_df["LongMovie"] = imdb_df["Runtime"] > 120

In [56]:
director_counts = imdb_df['Director'].value_counts()

famous_directors = director_counts[director_counts >= 4].index.tolist()

imdb_df['Famous Director'] = imdb_df['Director'].isin(famous_directors)

In [57]:
# Calculate the upper 10th percentile of numVotes
upper_10th_percentile = imdb_df['numVotes'].quantile(0.9)

# Create the SuperFamousMovie column based on the upper 10th percentile
imdb_df['SuperFamousMovie'] = imdb_df['numVotes'] > upper_10th_percentile

In [58]:
# Split the genres column into multiple columns
genre_columns = imdb_df['genres'].str.split(',', expand=True)

# Rename the genre columns
genre_columns.columns = ['Genre1', 'Genre2', 'Genre3']

# Concatenate the genre columns with the original dataframe
imdb_df = pd.concat([imdb_df, genre_columns], axis=1)

In [59]:
# Reorder the columns
imdb_df = imdb_df[['Const', 'Title', 'Director', 'Famous Director', 'ReleaseYear', 'Age', 'Runtime', 'LongMovie', 'genres', 'Genre1', 'Genre2', 'Genre3', 'numVotes', 'IMDb Rating',  'My Rating', 'DifferenceBetweenRatings', 'has_rated', 'Date Rated', 'lead_cast_1_name', 'lead_cast_2_name', 'lead_cast_3_name', 'SuperFamousMovie']]

In [60]:
# Create a seperate data frame for movies that the user has reviewed
reviewed_df = imdb_df[imdb_df['has_rated'] == True]

#### Create Dummy Columns for top directors

In [61]:
# Get the top 10 directors excluding "Multiple Directors" - Uses director counts from famousDirector column before
top_10_directors = director_counts[director_counts.index != 'Multiple Directors'].head(10)

# Get the top 10 directors excluding "Multiple Directors"
reviewed_director_counts = reviewed_df['Director'].value_counts()
top_10_reviewed_directors = reviewed_director_counts[reviewed_director_counts.index != 'Multiple Directors'].head(10)

# Creates dummy variables for the top 5 directors in reviewed movies and in all movies
for director in list(top_10_reviewed_directors.keys())[:5]:
    column_name = 'is_' + director.lower().replace(' ', '_')
    reviewed_df[column_name] = reviewed_df['Director'].str.lower().str.contains(director.lower())
    imdb_df[column_name] = imdb_df['Director'].str.lower().str.contains(director.lower())

for director in list(top_10_directors.keys())[:5]:
    column_name = 'is_' + director.lower().replace(' ', '_')
    reviewed_df[column_name] = reviewed_df['Director'].str.lower().str.contains(director.lower())
    imdb_df[column_name] = imdb_df['Director'].str.lower().str.contains(director.lower())

/tmp/ipykernel_1238084/2005626711.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewed_df[column_name] = reviewed_df['Director'].str.lower().str.contains(director.lower())
/tmp/ipykernel_1238084/2005626711.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewed_df[column_name] = reviewed_df['Director'].str.lower().str.contains(director.lower())
/tmp/ipykernel_1238084/2005626711.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [62]:
# Create dummies for top genres
genre_group = reviewed_df.groupby('Genre1')
genre_summary1 = genre_group.agg({'Genre1': 'count', 'IMDb Rating': 'sum', "My Rating": "sum"})
genre_summary1.columns = ['Genre Count', 'Total IMDb Rating', 'Total My Rating']

genre_group = reviewed_df.groupby('Genre2')
genre_summary2 = genre_group.agg({'Genre2': 'count', 'IMDb Rating': 'sum', "My Rating": "sum"})
genre_summary2.columns = ['Genre Count', 'Total IMDb Rating', 'Total My Rating']

genre_group = reviewed_df.groupby('Genre3')
genre_summary3 = genre_group.agg({'Genre3': 'count', 'IMDb Rating': 'sum', "My Rating": "sum"})
genre_summary3.columns = ['Genre Count', 'Total IMDb Rating', 'Total My Rating']

reviewed_genre_summary = pd.concat([genre_summary1, genre_summary2, genre_summary3])
reviewed_genre_summary = reviewed_genre_summary.groupby(level=0).sum()
reviewed_genre_summary["Average Rating"] = reviewed_genre_summary["Total IMDb Rating"] / reviewed_genre_summary["Genre Count"]
reviewed_genre_summary["My Average Rating"] = reviewed_genre_summary["Total My Rating"] / reviewed_genre_summary["Genre Count"]
reviewed_genre_summary["Difference"] = reviewed_genre_summary["My Average Rating"] - reviewed_genre_summary["Average Rating"]

# After making a summary table for the genres, this code creates dummy variables for genres 
# with an absolute difference of more than 0.5 or an average rating of more than 8
for genre in reviewed_genre_summary.index:
    difference = reviewed_genre_summary.loc[genre, 'Difference']
    my_average_rating = reviewed_genre_summary.loc[genre, 'My Average Rating']
    
    if abs(difference) > 0.5 or my_average_rating > 8:
        column_name = 'is' + genre.lower().replace(' ', '_')
        reviewed_df[column_name] = reviewed_df['genres'].str.lower().str.contains(genre.lower())
        imdb_df[column_name] = imdb_df['genres'].str.lower().str.contains(genre.lower())

/tmp/ipykernel_1238084/2559324673.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewed_df[column_name] = reviewed_df['genres'].str.lower().str.contains(genre.lower())
/tmp/ipykernel_1238084/2559324673.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewed_df[column_name] = reviewed_df['genres'].str.lower().str.contains(genre.lower())
/tmp/ipykernel_1238084/2559324673.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [63]:
# Create a list of all unique lead actors
all_lead_actors = pd.concat([reviewed_df['lead_cast_1_name'], reviewed_df['lead_cast_2_name'], reviewed_df['lead_cast_3_name']]).unique()

# Initialize the temporary dataframe with columns for each lead actor
temp_lead_actors_df = pd.DataFrame(0, index=reviewed_df.index, columns=all_lead_actors)

# Iterate through each movie and add the "My Rating" to the corresponding lead actor columns, ensuring each actor is only counted once per movie
for idx, row in reviewed_df.iterrows():
    actors_seen = set()
    for lead_actor in ['lead_cast_1_name', 'lead_cast_2_name', 'lead_cast_3_name']:
        actor_name = row[lead_actor]
        if pd.notna(actor_name) and actor_name not in actors_seen:
            temp_lead_actors_df.at[idx, actor_name] += row['My Rating']
            actors_seen.add(actor_name)

# Calculate the total ratings and total observations for each lead actor
total_lead_actor_ratings = temp_lead_actors_df.sum()
total_lead_actor_observations = (temp_lead_actors_df > 0).sum()

# Calculate the average rating for each lead actor
average_lead_actor_ratings = total_lead_actor_ratings / total_lead_actor_observations

# Apply a weighting factor to reward actors who have been in more movies
weighting_factor = np.log1p(total_lead_actor_observations)  # Using log1p to avoid zero and smooth the effect
weighted_average_ratings = average_lead_actor_ratings * weighting_factor

# Normalize the weighted average ratings to fit within the range of 1 to 10
min_rating = weighted_average_ratings.min()
max_rating = weighted_average_ratings.max()
normalized_weighted_ratings = 1 + 9 * (weighted_average_ratings - min_rating) / (max_rating - min_rating)

# Get the top 7 best actors and bottom 3 worst actors based on normalized weighted average rating
top_7_best_actors = normalized_weighted_ratings.nlargest(7)
bottom_3_worst_actors = normalized_weighted_ratings.nsmallest(3)

# Create dummy variables for each actor in the top 7 best actors and bottom 3 worst actors list
for actor in top_7_best_actors.index.tolist() + bottom_3_worst_actors.index.tolist():
    column_name = 'is_' + actor.lower().replace(' ', '_')
    reviewed_df[column_name] = reviewed_df[['lead_cast_1_name', 'lead_cast_2_name', 'lead_cast_3_name']].apply(lambda x: actor in x.values, axis=1)
    imdb_df[column_name] = imdb_df[['lead_cast_1_name', 'lead_cast_2_name', 'lead_cast_3_name']].apply(lambda x: actor in x.values, axis=1)

/tmp/ipykernel_1238084/1273225276.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewed_df[column_name] = reviewed_df[['lead_cast_1_name', 'lead_cast_2_name', 'lead_cast_3_name']].apply(lambda x: actor in x.values, axis=1)
/tmp/ipykernel_1238084/1273225276.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewed_df[column_name] = reviewed_df[['lead_cast_1_name', 'lead_cast_2_name', 'lead_cast_3_name']].apply(lambda x: actor in x.values, axis=1)
/tmp/ipykernel_1238084/1273225276.py:39: SettingWi

In [64]:
if 'Age' in reviewed_df.columns:
    # Create dummy variables for age categories
    reviewed_df = pd.concat([reviewed_df, pd.get_dummies(pd.cut(reviewed_df['Age'], bins=[-1, 10, 25, np.inf], labels=['New', 'Old', 'Classic']), prefix='is')], axis=1)
if 'Age' in imdb_df.columns:
    # Create dummy variables for age categories
    imdb_df = pd.concat([imdb_df, pd.get_dummies(pd.cut(imdb_df['Age'], bins=[-1, 10, 25, np.inf], labels=['New', 'Old', 'Classic']), prefix='is')], axis=1)

## Below you can download both imdb_df and reviewed_df to be used for model building

In [65]:
imdb_df.to_csv('IMDB Model Data/imdb_data.csv', index=False)
reviewed_df.to_csv('IMDB Model Data/reviewed_data.csv', index=False)

In [66]:
reviewed_df.head()

,Const,Title,Director,Famous Director,ReleaseYear,Age,Runtime,LongMovie,genres,Genre1,...,is_chris_evans,is_christian_bale,is_chris_pratt,is_jonah_hill,is_rosalie_chiang,is_brie_larson,is_gemma_chan,is_New,is_Old,is_Classic
0,tt0111161,The Shawshank Redemption,Frank Darabont,False,1994,30,142,True,Drama,Drama,...,False,False,False,False,False,False,False,False,False,True
2,tt0468569,The Dark Knight,Christopher Nolan,True,2008,16,152,True,"Action,Crime,Drama",Action,...,False,True,False,False,False,False,False,False,True,False
7,tt0110912,Pulp Fiction,Quentin Tarantino,True,1994,30,154,True,"Crime,Drama",Crime,...,False,False,False,False,False,False,False,False,False,True
9,tt1375666,Inception,Christopher Nolan,True,2010,14,148,True,"Action,Adventure,Sci-Fi",Action,...,False,False,False,False,False,False,False,False,True,False
10,tt0137523,Fight Club,David Fincher,True,1999,25,139,True,Drama,Drama,...,False,False,False,False,False,False,False,False,True,False
